Run after dbCAN if there are errors due to formatting of the .gff file. These are common if there is an extra semicolon

In [14]:
import pandas as pd
import os, shutil

In [15]:
outfile_d = '/home/ecutts/dbCAN/outlogs/'
os.chdir(outfile_d)
problems = []
names = []
for f in os.listdir():
    if f.startswith('run_dbcan2'):
        file = open(f, "r")
        count = 0
        for aline in file:
            if count > 0:
                isPrinting = True
            else:
                isPrinting = False
            if aline.startswith('Traceback'): # this means there was an error
                print(f)
                print(aline)
                problems.append(f)
                count = 4
            if aline.endswith('.faa'):
                names.append(aline)
            if isPrinting:
                print(aline)
            count = count - 1

run_dbcan2_1.out
Traceback (most recent call last):

  File "run_dbcan.py", line 435, in <module>

    notes[temp[0]] = temp[1]

IndexError: list index out of range

run_dbcan2_3.out
Traceback (most recent call last):

  File "run_dbcan.py", line 435, in <module>

    notes[temp[0]] = temp[1]

IndexError: list index out of range

run_dbcan2_5.out
Traceback (most recent call last):

  File "run_dbcan.py", line 435, in <module>

    notes[temp[0]] = temp[1]

IndexError: list index out of range

run_dbcan2_6.out
Traceback (most recent call last):

  File "run_dbcan.py", line 435, in <module>

    notes[temp[0]] = temp[1]

IndexError: list index out of range

run_dbcan2_7.out
Traceback (most recent call last):

  File "run_dbcan.py", line 435, in <module>

    notes[temp[0]] = temp[1]

IndexError: list index out of range

run_dbcan2_9.out
Traceback (most recent call last):

  File "run_dbcan.py", line 435, in <module>

    notes[temp[0]] = temp[1]

IndexError: list index out of range

run_

In [23]:
#problems.remove('run_dbcan_3.out') # fixed this one manually.

# find and replace the rows in the gff file that have a stray semicolon.
# may need to be edited more specifically depending on your issues.
os.chdir('/home/ecutts/dbCAN')
genomes = pd.read_table('input/magslist.txt', header=None)
for p in problems:
    num = int(p.split('_')[2].split('.out')[0]) -1 
    genome = str(genomes.at[num,0])
    print(genome)
    gff = open('./input/' + genome + '.gff', "r")
    newgff = open('./input/' + genome + '_new.gff', "w")
    for line in gff:
        if line.startswith('#'):
            newgff.write(line)
        else:
            row = line.rstrip().split('\t')
            if row[2] == "CDS":
                note = row[8].split(";")
                gene = ""
                notes = {}
                for x in note:
                    temp = x.split('=')
                    try:
                        notes[temp[0]] = temp[1]
                        newgff.write(line)
                    except:
                        print(line)
                        newline = line.replace('; ', ', ')
                        newgff.write(line)

    os.remove('./input/' + genome + '.gff')
    os.rename('./input/' + genome + '_new.gff', './input/' + genome + '.gff')

2806310565
2806310567
2806310581
2806310582
2806310583
2806310585
2806310584


In [26]:
# make list of inputs, gffs, and filenames for running dbCAN:
inputs = open('./input/inputs3.txt', 'w')
gffs = open('./input/gffs3.txt', "w")
magslist = open('./input/magslist3.txt', "w")

for p in problems:
    num = int(p.split('_')[2].split('.out')[0]) -1 
    genome = str(genomes.at[num,0])
    print(genome)
    inputs.write(genome + '.genes.faa' + '\n')
    gffs.write(genome + '.gff' + '\n')
    magslist.write(genome + '\n')

2806310565
2806310567
2806310581
2806310582
2806310583
2806310585
2806310584


In [25]:
print(len(problems))

7


In [ ]:
# go edit sbatch script to rerun the problem files.